# 단순 선형 회귀
## 1. 패키지 준비

In [ ]:
import sys
sys.path.append('../../')
import helper

from pandas import read_excel, DataFrame
from matplotlib import pyplot as plt
import seaborn as sb
import numpy as np

from sklearn.model_selection import train_test_split # 데이터를 훈련용과 테스트용으로 나누는 기능
from tensorflow.keras.models import Sequential       # 순서층을 구성하는 모델 객체 생성 기능
from tensorflow.keras.layers import Dense            # 모델 객체에 학습층을 쌓기 위한 클래스

## 2. 데이터셋 준비

In [ ]:
origin = read_excel('https://data.hossam.kr/E04/cars.xlsx')
origin.head()

## 3. 데이터 전처리
- 결측치 확인

In [ ]:
origin.isna().sum()

> 결측치 없음

## 4. 탐색적 데이터 분석
- 기본 통계 확인

In [ ]:
origin.describe()

- 산점도 그래프와 추세선 확인

In [ ]:
plt.figure(figsize = (10, 5))
sb.regplot(data=origin,
           x='speed',
           y='dist',
           color= 'orange')
plt.grid()
plt.show()
plt.close()

> 곡선보다는 `선형분포`에 더 가까우므로 `단순선형회귀모델`을 적용하기로 결정

## 5. 데이터셋 분할
- 랜덤 시드 고정

In [ ]:
np.random.seed(777)

- 훈련 데이터(독립변수)와 레이블(종속변수) 구분하기

In [ ]:
x = origin.drop(['dist'], axis=1)
y = origin['dist']
print('훈련데이터 크기:', x.shape, ' / 레이블 크기:', y.shape)

- 훈련 데이터와 검증 데이터로 분할

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, 
                                                    y, 
                                                    test_size=0.3, 
                                                    random_state=777)

print('훈련용 데이터셋 크기: %d, 검증용 데이터셋 크기: %d' % (len(x_train), len(x_test)))

## 6. (학습 )모델 개발
- 모델 정의

In [ ]:
my_model = Sequential()

# 1차원의 데이터를 입력으로 받고, 32개의 출력을 가지는 첫 번째 Dense층
my_model.add(Dense(32, 
                   activation = 'relu',
                   input_shape = (1,)))
# 하나의 값을 출력
# -> 정답의 범위가 정해지지 않기 때문에 확성화 함수는 linear
# -> linear는 기본값이므로 생략 가능함
my_model.add(Dense(1, activation='linear'))

my_model.compile(optimizer = 'adam',
                 loss = 'mse',
                 metrics = ['mae'])
my_model.summary()

- 학습하기
    - loss = 훈련 데이터 손실률
    - mae = 훈련 절대오차
    - val_loss = 검증 데이터 손실률
    - val_mae = 검증 절대오차

In [ ]:
result = my_model.fit(x_train, y_train, epochs=500, validation_data=(x_test, y_test))

# 학습 결과
result_df = DataFrame(result.history)
result_df['epochs'] = result_df.index+1
result_df.set_index('epochs', inplace=True)
result_df

## 7. 학습 결과 평가
- 학습 결과 시각화

In [ ]:
# 그래프 기본 설정
plt.rcParams['font.family'] = 'AppleGothic'
plt.rcParams['font.size'] = 12
plt.rcParams['axes.unicode_minus'] = False

# 그래프를 그리기 위한 객체 생성
fig, (ax1, ax2) = plt.subplots(1, 2, figsize = (15, 5), dpi=150)

# 1) 훈련 및 검증 손실 그리기
sb.lineplot(x=result_df.index, y='loss', data=result_df, color='blue', label='훈련 손실률', ax=ax1)
sb.lineplot(x=result_df.index, y='val_loss', data=result_df, color='orange', label='검증 손실률', ax=ax1)
ax1.set_title('훈련 및 검증 손실률')
ax1.set_xlabel('반복회차')
ax1.set_ylabel('손실률')
ax1.grid()
ax1.legend()

# 2) 훈련 및 검증 절대오차 그리기
sb.lineplot(x=result_df.index, y='mae', data=result_df, color='blue', label='훈련 절대오차', ax=ax2)
sb.lineplot(x=result_df.index, y='val_mae', data=result_df, color='orange', label='검증 절대오차', ax=ax2)
ax2.set_title('훈련 및 검증 절대오차')
ax2.set_xlabel('반복회차')
ax2.set_ylabel('정확도')
ax2.grid()
ax2.legend()

plt.show()
plt.close()

- 모델 성능 평가

In [ ]:
evaluate1 = my_model.evaluate(x_train, y_train)
print('최종 훈련 손실룰: %f, 최종 훈련 절대오차: %f' % (evaluate1[0], evaluate1[1]))

evaluate2 = my_model.evaluate(x_test, y_test)
print('최종 검증 손실률: %f, 최종 검증 절대오차: %f' % (evaluate2[0], evaluate2[1]))

## 8. 학습 결과 적용
- 테스트 데이터에 대한 `예측 결과 산정`

In [ ]:
results = my_model.predict(x_test)
print(results)

- `결과 데이터셋 구성`

In [ ]:
kdf = DataFrame({'검증데이터': x_test['speed'],
                 '실제값': y_test,
                 '예측값': results.flatten()})

kdf['예측오차'] = kdf['실제값'] - kdf['예측값']
kdf

- `실제 결과값`과 `머신러닝에 의한 예측값` 비교

In [ ]:
helper.regplot(x_left=kdf['검증데이터'],
               y_left=kdf['실제값'],
               y_left_pred=kdf['예측값'])

- `임의의 값`에 대한 `머신러닝 예측 결과`

In [ ]:
my_speed = 50
results = my_model.predict([[my_speed]])
print('속도가 %d인 자동차의 제동거리는 %d로 예상됩니다.' % (my_speed, results))